In [ ]:
from moviepy.editor import VideoFileClip
import numpy as np
import cv2
import tkinter.filedialog as tkfd
from lesson_functions import *
from scipy.ndimage.measurements import label

if __name__ == "__main__":
    # Read movie
    #cap = cv2.VideoCapture('./project_video.mp4')
    cap = cv2.VideoCapture('./000011_2.mp4')

    # Buffer initialization
    ret, frame = cap.read()

    # Convert setting
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('vehicle_detection_result.avi',fourcc, 30.0, (frame.shape[1],frame.shape[0]))


    box_tmp_1 = []
    box_tmp_2 = []
    box_tmp_3 = []
    box_tmp_4 = []
    box_tmp_5 = []
    box_tmp = []
    boxes = []
    heat = []

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:

            #######################################################################################################
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            ystart = 400
            ystop = 656

            scale = 1.5
            window = 64    
            cells_per_step = 1      
            out_img, boxes_1 = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, window, cells_per_step)

            scale = 1
            window = 64    
            cells_per_step = 1      
            out_img, boxes_2 = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, window, cells_per_step)

            scale = 2
            window = 128    
            cells_per_step = 1      
            out_img, boxes_3 = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, window, cells_per_step)   

            boxes = boxes_1 + boxes_2 + boxes_3        

            #######################################################################################################     

            # Read in a pickle file with bboxes saved
            # Each item in the "all_bboxes" list will contain a 
            # list of boxes for one of the images shown above
            box_list = boxes
            box_tmp_5 = box_tmp_4
            box_tmp_4 = box_tmp_3
            box_tmp_3 = box_tmp_2
            box_tmp_2 = box_tmp_1
            box_tmp_1 = box_list
            box_tmp = box_tmp_5 + box_tmp_4 + box_tmp_3 + box_tmp_2 + box_tmp_1    

            # Read in image similar to one shown above 
            heat = np.zeros_like(image[:,:,0]).astype(np.float)

            # Add heat to each box in box list
            heat = add_heat(heat,box_tmp)        

            # Apply threshold to help remove false positives
            heat = apply_threshold(heat,10)

            # Visualize the heatmap when displaying    
            heatmap = np.clip(heat, 0, 255)

            # Find final boxes from heatmap using label function
            labels = label(heatmap)
            draw_img = draw_labeled_bboxes(np.copy(frame), labels)

            #######################################################################################################          
            showed_image = draw_img

            # Show movie
            out.write(showed_image)
            cv2.imshow('frame', showed_image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()